## Import the data and neccessary packages

In [2]:
import pandas as pd
import numpy as np

In [3]:
da_univ = pd.read_csv('PG University&College List - Ct_nh_ma_schools - Sheet1.csv')
da_store = pd.read_csv('Pg restaurant list 11.9.18 - Sheet1 - Pg restaurant list 11.9.18 - Sheet1.csv')

### Clean the data for analysis

In [4]:
# only show the relevant columns within the university data
da_univ.drop(da_univ.columns.difference(['INSTNM', 'LATITUDE', 'LONGITUDE', 'ZIP', 'CITY', 'STABBR']), 1, inplace=True)


# drop NA from the address column of the D'Angelo store locations
da_store = da_store.dropna(subset=['Address'])
da_store.drop(da_store.columns.difference(['Restaurant #', 'Restaurant Name', 'Address', 'City', 'State', 'Zip Code']), 1, inplace=True)
da_store.reset_index(drop=True)

# drop NA from the lat, lng column of the university data
da_univ = da_univ.dropna(subset=['LATITUDE', 'LONGITUDE'])



### Get the address of each university or college from lat long coordinates in university/college data.

In [5]:
# joined address, city, state colums by a comma into the address column
cols = ['Address', 'City', 'State']
da_store['Address'] = da_store[cols].apply(lambda row: ', '.join(row.values.astype(str)), axis=1)
da_store = da_store.reset_index(drop = True)
#da_store.head()

#### Use OpenCageGeocode

In [6]:
from opencage.geocoder import OpenCageGeocode


In [7]:
key = 'e48659b4500a4db7bf6fa31641f8b688'  # get api key from:  https://opencagedata.com



geocoder = OpenCageGeocode(key)




query = 'Denver, Colorado'  




results = geocoder.geocode(query)




#print (results)

In [227]:
lat = results[0]['geometry']['lat']

lng = results[0]['geometry']['lng']

print (lat, lng)

39.7392364 -104.9848623


## Grab lat, lng coordinates for each address within store data

In [8]:
list_lat = []   # create empty lists

list_long = []

for index, row in da_store.iterrows(): # iterate over rows in dataframe


    Address = row['Address']
    query = str(Address)

    results = geocoder.geocode(query)   
    lat = results[0]['geometry']['lat']
    long = results[0]['geometry']['lng']

    list_lat.append(lat)
    list_long.append(long)

# create new columns from lists    

da_store['lat'] = list_lat   

da_store['lon'] = list_long

In [9]:
da_store.head(85)
export_csv = da_store.to_csv('pg_store.csv')

In [10]:
da_store.shape

(97, 6)

In [11]:
da_univ.shape

(120, 6)

#### Haversine numpy function calculating distance between to lat, lng coordinate pairs

In [12]:

from math import radians, cos, sin, asin, sqrt
def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

In [241]:
# list_store = []
# dist = []


# ndx_stores = 0
# for index_store, row_store in da_store.iterrows():
#     store_lat = row_store['lat']
#     store_lon = row_store['lon']
    
#     ndx_univ = 0
#     for index_univ, row_univ in da_univ.iterrows():
#         univ_lat = row_univ['LATITUDE']
#         univ_lon = row_univ['LONGITUDE']
#         distance[ndx_univ] = haversine_np(store_lon, store_lat, univ_lon, univ_lat)
#         ndx_univ += 1     
    
#     list_store[ndx_stores] = min(distance)
    

#points = haversine_np(da_store['lon'], da_store['lat'], da_univ['LONGITUDE'], da_univ['LATITUDE'])

#print(points)
#points = points.reshape(1, -1)
#haversine_np(-97.441113,35.221090,-97.520280,35.465438)
#nbrs = NearestNeighbors(n_neighbors=2, metric=distance).fit(points)

#distances, indices = nbrs.kneighbors(points)

#result = distances[:, 1]
#da_store.head()

In [133]:
# export da_univ to csv
export_csv2 = da_univ.to_csv('da_univ.csv')

In [189]:
#%qtconsole

# Created dataframe with store, univ, restaurant name, INSTNM, and CLOSEST UNIVERSITY DISTANCE

In [13]:
# Build an index that contain every pairing of Store - University
idx = pd.MultiIndex.from_product([da_store.index, da_univ.index], names=['Store', 'Univ'])

# Pull the coordinates of the store and the universities together
# We don't need their name here
df = pd.DataFrame(index=idx) \
        .join(da_store[['lat', 'lon']], on='Store') \
        .join(da_univ[['LATITUDE', 'LONGITUDE']], on='Univ')

df['Distance'] = haversine_np(*df[['lat', 'lon', 'LATITUDE', 'LONGITUDE']].values.T)

# The closest university to each store
min_distance = df.loc[df.groupby('Store')['Distance'].idxmin(), 'Distance']

# Pulling everything together
df_comp = min_distance.to_frame().join(da_store, on='Store').join(da_univ, on='Univ') \
    [['Restaurant Name', 'INSTNM', 'Distance']]


df_comp.head()

,,Restaurant Name,INSTNM,Distance
Store,Univ,,,
0,86,Derry,Lincoln Technical Institute-Lowell,9.620209
1,73,Rockingham,EINE Inc,5.985114
2,113,Gilford,Lakes Region Community College,2.812440
3,107,"Rochester, NH (Lilac)",Empire Beauty School-Somersworth,10.910741
4,117,Concord,Granite State College,2.031957


In [14]:
# export final dataframe to csv
ex = df_comp.to_csv('df_comp_papagino.csv')